In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

## Creación de datos para exportación

In [ ]:
np.random.seed(42)

# Generar datos de una semana para múltiples pacientes
pacientes = ['P001', 'P002', 'P003', 'P004', 'P005']
inicio = pd.Timestamp('2025-01-01 00:00:00')
n_horas = 168  # 1 semana

datos = []
for paciente in pacientes:
    for hora in range(n_horas):
        timestamp = inicio + timedelta(hours=hora)
        datos.append({
            'paciente_id': paciente,
            'timestamp': timestamp,
            'fecha': timestamp.date(),
            'hora': timestamp.hour,
            'HR': np.random.randint(60, 100, 1)[0],
            'SBP': np.random.randint(100, 140, 1)[0],
            'DBP': np.random.randint(60, 90, 1)[0],
            'SpO2': np.random.randint(94, 100, 1)[0],
            'temperatura': np.round(np.random.uniform(36.5, 37.5), 1)
        })

df = pd.DataFrame(datos)

print("=== Dataset generado ===")
print(df.head(10))
print(f"\nTotal de registros: {len(df)}")
print(f"Pacientes: {df['paciente_id'].nunique()}")
print(f"Periodo: {df['timestamp'].min()} a {df['timestamp'].max()}")

## 1. Exportación básica a CSV

In [ ]:
# Exportar DataFrame completo a CSV
archivo_csv = 'signos_vitales_completo.csv'

df.to_csv(archivo_csv, index=False)
print(f"✓ Datos exportados a: {archivo_csv}")

# Verificar archivo
if os.path.exists(archivo_csv):
    tamaño = os.path.getsize(archivo_csv) / 1024  # KB
    print(f"  Tamaño: {tamaño:.2f} KB")

## 2. Exportación con opciones personalizadas

In [ ]:
# CSV con opciones personalizadas
archivo_csv_custom = 'signos_vitales_custom.csv'

df.to_csv(
    archivo_csv_custom,
    index=False,              # No exportar índice
    sep=';',                  # Separador punto y coma
    encoding='utf-8',         # Codificación UTF-8
    decimal=',',              # Decimal con coma
    float_format='%.2f'       # 2 decimales
)

print(f"✓ CSV personalizado exportado: {archivo_csv_custom}")
print("  Configuración: separador ';', decimal ','")

## 3. Exportar solo columnas seleccionadas

In [ ]:
# Exportar solo columnas relevantes
archivo_resumen = 'signos_vitales_resumen.csv'

columnas_exportar = ['paciente_id', 'timestamp', 'HR', 'SBP', 'SpO2']
df[columnas_exportar].to_csv(archivo_resumen, index=False)

print(f"✓ Resumen exportado: {archivo_resumen}")
print(f"  Columnas: {', '.join(columnas_exportar)}")

## 4. Exportación a Excel

In [ ]:
# Exportar a Excel básico
archivo_excel = 'signos_vitales.xlsx'

df.to_excel(archivo_excel, index=False, sheet_name='Signos Vitales')

print(f"✓ Datos exportados a Excel: {archivo_excel}")
if os.path.exists(archivo_excel):
    tamaño = os.path.getsize(archivo_excel) / 1024
    print(f"  Tamaño: {tamaño:.2f} KB")

## 5. Excel con múltiples hojas

In [ ]:
# Crear Excel con múltiples hojas
archivo_excel_multi = 'reporte_completo.xlsx'

with pd.ExcelWriter(archivo_excel_multi, engine='openpyxl') as writer:
    # Hoja 1: Datos completos
    df.to_excel(writer, sheet_name='Datos Completos', index=False)
    
    # Hoja 2: Resumen por paciente
    resumen_paciente = df.groupby('paciente_id').agg({
        'HR': ['mean', 'min', 'max', 'std'],
        'SBP': ['mean', 'min', 'max'],
        'SpO2': ['mean', 'min']
    }).round(2)
    resumen_paciente.to_excel(writer, sheet_name='Resumen por Paciente')
    
    # Hoja 3: Resumen diario
    resumen_diario = df.groupby(['paciente_id', 'fecha']).agg({
        'HR': 'mean',
        'SBP': 'mean',
        'SpO2': 'mean'
    }).round(2)
    resumen_diario.to_excel(writer, sheet_name='Resumen Diario')

print(f"✓ Excel multi-hoja creado: {archivo_excel_multi}")
print("  Hojas: Datos Completos, Resumen por Paciente, Resumen Diario")

## 6. Exportar tablas dinámicas

In [ ]:
# Crear y exportar tabla dinámica
tabla_pivot = pd.pivot_table(
    df,
    values='HR',
    index='paciente_id',
    columns='fecha',
    aggfunc='mean'
).round(2)

archivo_pivot = 'tabla_pivot_hr_diario.csv'
tabla_pivot.to_csv(archivo_pivot)

print(f"✓ Tabla dinámica exportada: {archivo_pivot}")
print(f"  Dimensiones: {tabla_pivot.shape[0]} pacientes x {tabla_pivot.shape[1]} días")
print("\nVista previa:")
print(tabla_pivot.head())

## 7. Exportar estadísticas resumidas

In [ ]:
# Crear tabla de estadísticas
estadisticas = df[['HR', 'SBP', 'DBP', 'SpO2', 'temperatura']].describe()

archivo_stats = 'estadisticas_descriptivas.csv'
estadisticas.to_csv(archivo_stats)

print(f"✓ Estadísticas exportadas: {archivo_stats}")
print("\nContenido:")
print(estadisticas)

## 8. Curvas suavizadas (rolling mean)

In [ ]:
# Calcular curvas suavizadas
df_suavizado = df.copy()
df_suavizado = df_suavizado.sort_values(['paciente_id', 'timestamp'])

# Para cada paciente, calcular rolling mean
ventana = 6  # 6 horas
df_suavizado['HR_suavizado'] = df_suavizado.groupby('paciente_id')['HR'].transform(
    lambda x: x.rolling(window=ventana, min_periods=1).mean()
)
df_suavizado['SBP_suavizado'] = df_suavizado.groupby('paciente_id')['SBP'].transform(
    lambda x: x.rolling(window=ventana, min_periods=1).mean()
)

# Exportar
archivo_suavizado = 'signos_vitales_suavizados.csv'
df_suavizado.to_csv(archivo_suavizado, index=False)

print(f"✓ Curvas suavizadas exportadas: {archivo_suavizado}")
print(f"  Ventana de suavizado: {ventana} horas")
print("\nVista previa:")
print(df_suavizado[['paciente_id', 'timestamp', 'HR', 'HR_suavizado', 'SBP', 'SBP_suavizado']].head(10))

## 9. Reporte semanal por paciente

In [ ]:
# Generar reporte semanal completo por paciente
def generar_reporte_semanal(paciente_id):
    datos_paciente = df[df['paciente_id'] == paciente_id].copy()
    
    # Calcular estadísticas
    reporte = pd.DataFrame({
        'Métrica': ['HR promedio', 'HR mínimo', 'HR máximo', 'HR std',
                    'SBP promedio', 'SBP mínimo', 'SBP máximo',
                    'DBP promedio', 'DBP mínimo', 'DBP máximo',
                    'SpO2 promedio', 'SpO2 mínimo',
                    'Temperatura promedio'],
        'Valor': [
            datos_paciente['HR'].mean(),
            datos_paciente['HR'].min(),
            datos_paciente['HR'].max(),
            datos_paciente['HR'].std(),
            datos_paciente['SBP'].mean(),
            datos_paciente['SBP'].min(),
            datos_paciente['SBP'].max(),
            datos_paciente['DBP'].mean(),
            datos_paciente['DBP'].min(),
            datos_paciente['DBP'].max(),
            datos_paciente['SpO2'].mean(),
            datos_paciente['SpO2'].min(),
            datos_paciente['temperatura'].mean()
        ]
    })
    
    reporte['Valor'] = reporte['Valor'].round(2)
    
    # Agregar metadatos
    reporte.loc[len(reporte)] = ['Total mediciones', len(datos_paciente)]
    reporte.loc[len(reporte)] = ['Periodo inicio', datos_paciente['timestamp'].min()]
    reporte.loc[len(reporte)] = ['Periodo fin', datos_paciente['timestamp'].max()]
    
    return reporte

# Generar reportes para todos los pacientes
archivo_reportes = 'reportes_semanales.xlsx'

with pd.ExcelWriter(archivo_reportes, engine='openpyxl') as writer:
    for paciente in pacientes:
        reporte = generar_reporte_semanal(paciente)
        reporte.to_excel(writer, sheet_name=paciente, index=False)
        print(f"✓ Reporte generado para {paciente}")

print(f"\n✓ Todos los reportes guardados en: {archivo_reportes}")

## 10. Exportar múltiples archivos por paciente

In [ ]:
# Crear carpeta para reportes individuales
carpeta_reportes = 'reportes_pacientes'
os.makedirs(carpeta_reportes, exist_ok=True)

# Generar archivo CSV para cada paciente
for paciente in pacientes:
    datos_paciente = df[df['paciente_id'] == paciente]
    archivo = os.path.join(carpeta_reportes, f'{paciente}_semana.csv')
    datos_paciente.to_csv(archivo, index=False)
    print(f"✓ {paciente}: {len(datos_paciente)} registros → {archivo}")

print(f"\n✓ Todos los archivos guardados en carpeta: {carpeta_reportes}/")

## 11. Exportar con formato de fecha personalizado

In [ ]:
# Crear DataFrame con formato de fecha personalizado
df_formato = df.copy()
df_formato['fecha_formateada'] = df_formato['timestamp'].dt.strftime('%d/%m/%Y %H:%M')

# Exportar
archivo_formato = 'signos_vitales_formato.csv'
df_formato[['paciente_id', 'fecha_formateada', 'HR', 'SBP', 'SpO2']].to_csv(
    archivo_formato, 
    index=False
)

print(f"✓ Archivo con formato de fecha personalizado: {archivo_formato}")
print("  Formato: DD/MM/YYYY HH:MM")
print("\nVista previa:")
print(df_formato[['paciente_id', 'fecha_formateada', 'HR']].head())

## 12. Crear reporte ejecutivo consolidado

In [ ]:
# Crear reporte ejecutivo consolidado
archivo_ejecutivo = 'reporte_ejecutivo_semanal.xlsx'

with pd.ExcelWriter(archivo_ejecutivo, engine='openpyxl') as writer:
    # HOJA 1: Resumen general
    resumen_general = pd.DataFrame({
        'Indicador': [
            'Total de pacientes',
            'Total de mediciones',
            'Periodo',
            'HR promedio general',
            'SBP promedio general',
            'SpO2 promedio general'
        ],
        'Valor': [
            df['paciente_id'].nunique(),
            len(df),
            f"{df['timestamp'].min()} - {df['timestamp'].max()}",
            f"{df['HR'].mean():.2f} bpm",
            f"{df['SBP'].mean():.2f} mmHg",
            f"{df['SpO2'].mean():.2f} %"
        ]
    })
    resumen_general.to_excel(writer, sheet_name='Resumen General', index=False)
    
    # HOJA 2: Comparativa por paciente
    comparativa = df.groupby('paciente_id').agg({
        'HR': 'mean',
        'SBP': 'mean',
        'DBP': 'mean',
        'SpO2': 'mean',
        'temperatura': 'mean'
    }).round(2)
    comparativa.columns = ['HR_promedio', 'SBP_promedio', 'DBP_promedio', 
                           'SpO2_promedio', 'Temp_promedio']
    comparativa.to_excel(writer, sheet_name='Comparativa Pacientes')
    
    # HOJA 3: Estadísticas descriptivas
    stats = df[['HR', 'SBP', 'DBP', 'SpO2']].describe()
    stats.to_excel(writer, sheet_name='Estadísticas Descriptivas')
    
    # HOJA 4: Tendencias diarias
    tendencia_diaria = df.groupby('fecha').agg({
        'HR': 'mean',
        'SBP': 'mean',
        'SpO2': 'mean'
    }).round(2)
    tendencia_diaria.to_excel(writer, sheet_name='Tendencias Diarias')
    
    # HOJA 5: Valores extremos
    extremos = pd.DataFrame({
        'Paciente': [],
        'Timestamp': [],
        'Tipo': [],
        'Valor': []
    })
    
    # HR máximos y mínimos por paciente
    for paciente in pacientes:
        datos_p = df[df['paciente_id'] == paciente]
        idx_max = datos_p['HR'].idxmax()
        idx_min = datos_p['HR'].idxmin()
        
        extremos = pd.concat([extremos, pd.DataFrame({
            'Paciente': [paciente, paciente],
            'Timestamp': [datos_p.loc[idx_max, 'timestamp'], datos_p.loc[idx_min, 'timestamp']],
            'Tipo': ['HR máximo', 'HR mínimo'],
            'Valor': [datos_p.loc[idx_max, 'HR'], datos_p.loc[idx_min, 'HR']]
        })], ignore_index=True)
    
    extremos.to_excel(writer, sheet_name='Valores Extremos', index=False)

print(f"✓ Reporte ejecutivo consolidado creado: {archivo_ejecutivo}")
print("\nHojas incluidas:")
print("  1. Resumen General")
print("  2. Comparativa Pacientes")
print("  3. Estadísticas Descriptivas")
print("  4. Tendencias Diarias")
print("  5. Valores Extremos")

## 13. Listado de archivos generados

In [ ]:
# Listar todos los archivos generados
print("=== ARCHIVOS GENERADOS ===")
print("\nCSV:")
archivos_csv = [
    'signos_vitales_completo.csv',
    'signos_vitales_custom.csv',
    'signos_vitales_resumen.csv',
    'tabla_pivot_hr_diario.csv',
    'estadisticas_descriptivas.csv',
    'signos_vitales_suavizados.csv',
    'signos_vitales_formato.csv'
]

for archivo in archivos_csv:
    if os.path.exists(archivo):
        tamaño = os.path.getsize(archivo) / 1024
        print(f"  ✓ {archivo} ({tamaño:.2f} KB)")

print("\nExcel:")
archivos_excel = [
    'signos_vitales.xlsx',
    'reporte_completo.xlsx',
    'reportes_semanales.xlsx',
    'reporte_ejecutivo_semanal.xlsx'
]

for archivo in archivos_excel:
    if os.path.exists(archivo):
        tamaño = os.path.getsize(archivo) / 1024
        print(f"  ✓ {archivo} ({tamaño:.2f} KB)")

print("\nCarpetas:")
if os.path.exists(carpeta_reportes):
    archivos_carpeta = os.listdir(carpeta_reportes)
    print(f"  ✓ {carpeta_reportes}/ ({len(archivos_carpeta)} archivos)")
    for archivo in archivos_carpeta[:3]:  # Mostrar primeros 3
        print(f"    - {archivo}")

print("\n" + "="*50)
print("✓ Todos los reportes generados exitosamente")

## Resumen

### Métodos de exportación:

1. **to_csv()**:
   - Rápido y ligero
   - Compatible universalmente
   - Opciones: separador, encoding, formato decimal
   ```python
   df.to_csv('archivo.csv', index=False, sep=';', encoding='utf-8')
   ```

2. **to_excel()**:
   - Mejor presentación
   - Múltiples hojas
   - Requiere openpyxl o xlsxwriter
   ```python
   df.to_excel('archivo.xlsx', sheet_name='Hoja1', index=False)
   ```

3. **ExcelWriter**:
   - Múltiples hojas en un archivo
   - Mayor control
   ```python
   with pd.ExcelWriter('archivo.xlsx') as writer:
       df1.to_excel(writer, sheet_name='Hoja1')
       df2.to_excel(writer, sheet_name='Hoja2')
   ```

### Tipos de reportes generados:

1. **Datos crudos**: Dataset completo
2. **Resúmenes estadísticos**: describe(), agg()
3. **Tablas dinámicas**: pivot_table()
4. **Curvas suavizadas**: rolling().mean()
5. **Reportes por entidad**: groupby() + exportación
6. **Reportes ejecutivos**: consolidación de múltiples análisis

### Buenas prácticas:

- ✓ Usar `index=False` si no es necesario
- ✓ Nombres de archivo descriptivos con fecha
- ✓ Organizar en carpetas por tipo/periodo
- ✓ Documentar formato y codificación
- ✓ Validar existencia de archivos generados
- ✓ Redondear decimales apropiadamente
- ✓ Incluir metadatos (fecha, periodo, versión)

### Aplicación clínica:
- Reportes semanales automáticos
- Compartir datos con equipo médico
- Auditorías y registros
- Análisis retrospectivos
- Documentación de resultados